In [1]:
import tensorflow as tf

# Check for available GPUs and enable memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPUs found and configured:", gpus)
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. Please set your runtime to GPU in Kaggle Notebook settings.")

GPUs found and configured: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
!pip install -q medmnist

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import f1_score
import medmnist
from medmnist import INFO
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

# MedMNIST images are 28x28. Here, we assume 3 channels.
TARGET_SIZE = (28, 28)
BATCH_SIZE = 32

In [4]:
def load_medmnist_data(task, split):
    """
    Load MedMNIST data for a given task and split ('train', 'val', or 'test').
    """
    DataClass = getattr(medmnist, INFO[task]['python_class'])
    ds = DataClass(split=split, download=True)
    return ds

def create_tf_dataset_from_medmnist(medmnist_ds, batch_size=BATCH_SIZE, augment=False):
    """
    Convert a MedMNIST dataset object (with attributes `imgs` and `labels`) into a tf.data.Dataset.
    We convert images to have 3 channels. (MedMNIST images are originally grayscale.)
    """
    x = medmnist_ds.imgs  # Expected shape: (N, 28, 28)
    y = medmnist_ds.labels.squeeze()  # Expected shape: (N,)
    
    # Expand dims to get (N,28,28,1), then tile to 3 channels
    if x.ndim == 3:
        x = np.expand_dims(x, axis=-1)  # (N,28,28,1)
    if x.shape[-1] == 1:
        x = np.tile(x, (1, 1, 1, 3))  # (N,28,28,3)
    
    x = x.astype(np.float32) / 255.0  # Normalize to [0,1]
    
    def _process(image, label):
        # Images are already 28x28; no resizing needed.
        if augment:
            image = tf.image.random_flip_left_right(image)
            image = tf.image.random_flip_up_down(image)
        return image, label
    
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(_process, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.shuffle(buffer_size=len(x))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [5]:
# Define the list of tasks (excluding ChestMNIST)
task_names = [
    "pathmnist",
    "dermamnist",
    "octmnist",
    "pneumoniamnist",
    "retinamnist",
    "breastmnist",
    "bloodmnist",
    "tissuemnist",
    "organamnist",
    "organcmnist",
    "organsmnist"
]

# Load datasets for each task
train_datasets = {}
val_datasets = {}
test_datasets = {}

for task in task_names:
    train_datasets[task] = load_medmnist_data(task, 'train')
    val_datasets[task] = load_medmnist_data(task, 'val')
    test_datasets[task] = load_medmnist_data(task, 'test')
    print(f"{task}: {len(train_datasets[task])} train, {len(val_datasets[task])} val, {len(test_datasets[task])} test samples")

100%|██████████| 206M/206M [00:16<00:00, 12.8MB/s]


Using downloaded and verified file: /root/.medmnist/pathmnist.npz
Using downloaded and verified file: /root/.medmnist/pathmnist.npz
pathmnist: 89996 train, 10004 val, 7180 test samples


100%|██████████| 19.7M/19.7M [00:02<00:00, 8.68MB/s]


Using downloaded and verified file: /root/.medmnist/dermamnist.npz
Using downloaded and verified file: /root/.medmnist/dermamnist.npz
dermamnist: 7007 train, 1003 val, 2005 test samples


100%|██████████| 54.9M/54.9M [00:05<00:00, 9.50MB/s]


...

Using downloaded and verified file: /root/.medmnist/organamnist.npz
Using downloaded and verified file: /root/.medmnist/organamnist.npz
organamnist: 34561 train, 6491 val, 17778 test samples


100%|██████████| 15.5M/15.5M [00:01<00:00, 7.84MB/s]


Using downloaded and verified file: /root/.medmnist/organcmnist.npz
Using downloaded and verified file: /root/.medmnist/organcmnist.npz
organcmnist: 12975 train, 2392 val, 8216 test samples


100%|██████████| 16.5M/16.5M [00:02<00:00, 7.82MB/s]


Using downloaded and verified file: /root/.medmnist/organsmnist.npz
Using downloaded and verified file: /root/.medmnist/organsmnist.npz
organsmnist: 13932 train, 2452 val, 8827 test samples


In [6]:
train_datasets_tf = {}
val_datasets_tf = {}
test_datasets_tf = {}

for task in task_names:
    train_datasets_tf[task] = create_tf_dataset_from_medmnist(train_datasets[task], batch_size=BATCH_SIZE, augment=True)
    val_datasets_tf[task] = create_tf_dataset_from_medmnist(val_datasets[task], batch_size=BATCH_SIZE, augment=False)
    test_datasets_tf[task] = create_tf_dataset_from_medmnist(test_datasets[task], batch_size=BATCH_SIZE, augment=False)
    
    # Optionally, count the number of batches in the test dataset
    count = sum(1 for _ in test_datasets_tf[task])
    print(f"{task}: Test batches: {count}")

pathmnist: Test batches: 225
dermamnist: Test batches: 63
octmnist: Test batches: 32
pneumoniamnist: Test batches: 20
retinamnist: Test batches: 13
breastmnist: Test batches: 5
bloodmnist: Test batches: 107
tissuemnist: Test batches: 1478
organamnist: Test batches: 556
organcmnist: Test batches: 257
organsmnist: Test batches: 276


In [7]:
def build_lenet_model(num_classes, input_shape=(28,28,3)):
    model = Sequential([
        Conv2D(6, (5,5), activation='relu', padding='same', input_shape=input_shape),
        AveragePooling2D(pool_size=(2,2)),
        Conv2D(16, (5,5), activation='relu'),
        AveragePooling2D(pool_size=(2,2)),
        Flatten(),
        Dense(120, activation='relu'),
        Dense(84, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
models = {}
histories = {}

for task in task_names:
    num_classes = len(INFO[task]['label'])
    print(f"\nTraining LeNet-5 model for {task} with {num_classes} classes...")
    model_task = build_lenet_model(num_classes, input_shape=(28,28,3))
    history = model_task.fit(train_datasets_tf[task],
                             validation_data=val_datasets_tf[task],
                             epochs=5,   # Increase epochs for better performance
                             verbose=1)
    models[task] = model_task
    histories[task] = history


Training LeNet-5 model for pathmnist with 9 classes...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.3675 - loss: 1.6481 - val_accuracy: 0.5906 - val_loss: 1.0661
Epoch 2/5
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.6083 - loss: 1.0274 - val_accuracy: 0.6364 - val_loss: 0.9511
Epoch 3/5
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.6570 - loss: 0.9149 - val_accuracy: 0.6993 - val_loss: 0.8027
Epoch 4/5
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.6822 - loss: 0.8434 - val_accuracy: 0.6795 - val_loss: 0.8785
Epoch 5/5
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.7058 - loss: 0.7844 - val_accuracy: 0.7435 - val_loss: 0.6940

Training LeNet-5 model for dermamnist with 7 classes...
Epoch 1/5
219/219 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.6473 - loss: 1.1477 - val_accuracy: 0.6780 - val_loss: 0.9366
Epoch 2/5
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6826 - loss: 0.9203 - val_accuracy: 0.6800 - val_loss: 0.9350
Epoch 3/5
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 

In [9]:
submission_rows = []
global_id = 0

for task in task_names:
    model_task = models[task]
    preds_list = []
    for images, _ in test_datasets_tf[task]:
        preds = model_task.predict(images)
        preds = np.argmax(preds, axis=1)  # Convert softmax outputs to class labels
        preds_list.append(preds)
    preds_all = np.concatenate(preds_list)
    for idx, pred in enumerate(preds_all):
        submission_rows.append([global_id, idx, task, int(pred)])
        global_id += 1

submission_df = pd.DataFrame(submission_rows, columns=["id", "id_image_in_task", "task_name", "label"])
print("Total submission rows:", len(submission_df))
submission_df.to_csv("submission.csv", index=False)
print("Submission file saved as submission.csv")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━